In [3]:
import json
import textwrap
import pandas as pd
from collections import defaultdict
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

c:\Users\moham\narrative_extraction\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset_path = "urdu-news-dataset-clean.csv"
df = pd.read_csv(dataset_path, encoding="utf-8")

print(df.head())

MODEL_NAME = "mirfan899/urdu-bert-ner" 

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME)

ner_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple" 
)

   Index                                           Headline  \
0      0  عالمی بینک عسکریت پسندی سے متاثرہ خاندانوں کی ...   
1      1  مالی سال 2020 ریٹرن فائل کرنے والوں کی تعداد م...   
2      2  جاپان کو سندھ کے خصوصی اقتصادی زون میں سرمایہ ...   
3      3   برامدات 767 فیصد بڑھ کر ارب 16 کروڑ ڈالر سے زائد   
4      4  کے الیکٹرک کو اضافی بجلی گیس کی فراہمی کے قانو...   

                                           News Text              Category  \
0  اسلام باد عالمی بینک خیبرپختونخوا کے قبائلی اض...  Business & Economics   
1  اسلام باد فیڈرل بورڈ ریونیو ایف بی نے دسمبر کی...  Business & Economics   
2  اسلام باد بورڈ انویسٹمنٹ بی او ئی کے چیئرمین ع...  Business & Economics   
3  اسلام اباد پاکستان میں ماہ نومبر میں مسلسل تیس...  Business & Economics   
4  اسلام باد نیشنل ٹرانسمیشن اینڈ ڈسپیچ کمپنی این...  Business & Economics   

         Date                                    URL     Source  News length  
0  2020-12-06  https://www.dawnnews.tv/news/1148499/  Dawn News       185

Device set to use cpu


In [5]:
def chunk_text(text, max_length=450):
    return textwrap.wrap(text, width=max_length)

def extract_named_entities(text):
    if not text or not isinstance(text, str):
        return {}

    chunks = chunk_text(text)  
    all_entities = []

    for chunk in chunks:
        entities = ner_pipeline(chunk) 
        all_entities.extend(entities) 

    structured_entities = defaultdict(list)
    
    for entity in all_entities:
        entity_type = entity.get("entity_group", entity.get("entity", "UNKNOWN"))  # Handle variations
        entity_text = entity["word"].replace("##", "")  

        if entity_text not in structured_entities[entity_type]:  
            structured_entities[entity_type].append(entity_text)

    return dict(structured_entities)

ner_results = []
for i in range(100): 
    row = df.iloc[i]  
    text_content = row.get("News Text", "").strip() 
    
    if not text_content:
        continue 

    entities = extract_named_entities(text_content)
    ner_results.append({
        "headline": row.get("Headline", "No Headline"),  
        "category": row.get("Category", "Unknown"),  
        "date": row.get("Date", "Unknown"),  
        "source": row.get("Source", "Unknown"),  
        "entities": entities
    })

with open("urdu.json", "w", encoding="utf-8") as f:
    json.dump(ner_results, f, ensure_ascii=False, indent=4)